In [65]:
import pandas as pd
import requests as rq
import json
from io import StringIO
from datetime import timedelta, date, datetime
import numpy as np
from sklearn import preprocessing
import plotly.graph_objects as go
import statistics


# Stocks Fundamentals

In [2]:
data = rq.get("https://eodhistoricaldata.com/api/exchanges/US?api_token=&fmt=json")


In [3]:
symbols = json.loads(data.content)

In [4]:
stocks = [x for x in symbols if x['Type'] == 'Common Stock' and x['Exchange'] == "NYSE"]
print(len(stocks))


2713


In [5]:
stocks_df = pd.DataFrame(columns=["Code", "Exchange", "Name", "Sector", "Industry", "GicSector", "GicIndustry", "GicSubIndustry"])
stocks_fundamentals = []
counter = 0
for s in stocks:
    fun = rq.get("https://eodhistoricaldata.com/api/fundamentals/"+s["Code"]+"."+"US?api_token=&fmt=csv")
    fun = json.loads(fun.content)
    stocks_fundamentals.append(fun)
    #stocks_df.append({"Code": fun["General"]["Code"],"Exchange": fun["General"]["Exchange"],"Name": fun["General"]["Name"],"Sector": fun["General"]["Sector"],"Industry": fun["General"]["Industry"],"GicSector": fun["General"]["GicSector"],"GicIndustry": fun["General"]["GicIndustry"],"GicSubIndustry": fun["General"]["GicSubIndustry"] }, ignore_index=True)
    counter += 1
    print(counter)
    
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


In [6]:
with open('stocks_fundamentals-nyse.json', 'w') as f:
    json.dump(stocks_fundamentals, f, ensure_ascii=False)

## Fundamentals Metrics

In [69]:
with open('stocks_fundamentals-nyse.json') as f:
    new_stock_fundamentals = json.load(f)


In [70]:
by_gicIndustry = {}
#print(new_stock_fundamentals[0])
for s in new_stock_fundamentals:
    industry = s["General"]["GicSector"]
    if industry not in by_gicIndustry:
        by_gicIndustry[industry] = []
    by_gicIndustry[industry].append(s["General"]["Code"])

### Price Per Earning (P/E)

In [71]:
## get sector pe average
pe_sum = 0
count = 0
industries_pe_ratio = {}
worst_pe_ratios = {}
for i in by_gicIndustry:
    pe_sum = 0
    count = 0
    stocks = [s for s in new_stock_fundamentals if s['General']['Code'] in by_gicIndustry[i]]
    ratios = [r['Highlights']['PERatio'] for r in stocks if r['Highlights']['PERatio'] is not None and r['Highlights']['PERatio'] > 0 and r['Highlights']['PERatio'] < 300]
    industries_pe_ratio[i] = statistics.mean(ratios)
    #print([c['General']['Code'] for c in stocks])
    #print(industries_pe_ratio)

## find companies with worst pe ratio
for i in by_gicIndustry:
    #print(i)
    ratio_threshold = industries_pe_ratio[i] - industries_pe_ratio[i] * 0.50
    worst = [s for s in new_stock_fundamentals if s['General']['Code'] in by_gicIndustry[i]]
    worst = [{'symbol': e['General']['Code'], 'pe_ratio': e['Highlights']['PERatio']} for e in worst if e['Highlights']['PERatio'] is not None and e['Highlights']['PERatio'] < ratio_threshold]
    #print([c['General']['Code'] for c in worst])
    worst_pe_ratios[i] = worst
    


## Price per Book (P/B) ratio

In [72]:
## get sector pe average
pe_sum = 0
count = 0
worst_pb_ratios = {}
industries_pb_ratio = {}
ratios = []
#print(ratios)
## get mean pb ratio by industry
for i in by_gicIndustry:
    pe_sum = 0
    count = 0
    stocks = [s for s in new_stock_fundamentals if s['General']['Code'] in by_gicIndustry[i]]
    ratios = [r['Valuation']['PriceBookMRQ'] for r in stocks if r['Valuation']['PriceBookMRQ'] is not None and type(r['Valuation']['PriceBookMRQ']) is float]
    industries_pb_ratio[i] = statistics.mean(ratios)

#print(industries_pb_ratio)
## find companies with worst pb ratio
for i in by_gicIndustry:
    worst = []
    worst = [s for s['Valuation']['PriceBookMRQ'] in new_stock_fundamentals if s['General']['Code'] in by_gicIndustry[i]]
   # print()

    #print(i)
    ratio_threshold = industries_pb_ratio[i] - industries_pb_ratio[i] * 0.50
    worst = [s for s in new_stock_fundamentals if s['General']['Code'] in by_gicIndustry[i]]
    worst = [{'symbol': e['General']['Code'], 'pb_ratio': e['Valuation']['PriceBookMRQ']} for e in worst if type(e['Valuation']['PriceBookMRQ']) is float and e['Valuation']['PriceBookMRQ'] is not None and e['Valuation']['PriceBookMRQ'] < ratio_threshold]
    #print([c['General']['Code'] for c in worst])
    worst_pb_ratios[i] = worst
   # print(worst)


#### Find intersection

In [73]:
underperforming_symbols = []
for k in worst_pe_ratios.keys():
    industries_pe_ratio_mean = industries_pe_ratio[k]
    for s_pe in worst_pe_ratios[k]:
        for s_pb in worst_pb_ratios[k]:
            if s_pe['symbol'] == s_pb['symbol']:
                underperforming_symbols.append({'symbol': s_pe['symbol'], 'industry': k, 'pe_ratio': s_pe['pe_ratio'], 'industry_pe_ratio': industries_pe_ratio_mean, 'pb_ratio': s_pb['pb_ratio'], 'industry_pb_ratio': industries_pb_ratio[k]})
print(underperforming_symbols)



[{'symbol': 'ACB', 'industry': 'Health Care', 'pe_ratio': 3.1602, 'industry_pe_ratio': 44.649170103092786, 'pb_ratio': 0.3854, 'industry_pb_ratio': 10.13291447368421}, {'symbol': 'ANTM', 'industry': 'Health Care', 'pe_ratio': 12.2908, 'industry_pe_ratio': 44.649170103092786, 'pb_ratio': 2.0671, 'industry_pb_ratio': 10.13291447368421}, {'symbol': 'BIO', 'industry': 'Health Care', 'pe_ratio': 7.694, 'industry_pe_ratio': 44.649170103092786, 'pb_ratio': 2.0032, 'industry_pb_ratio': 10.13291447368421}, {'symbol': 'BKD', 'industry': 'Health Care', 'pe_ratio': 6.3043, 'industry_pe_ratio': 44.649170103092786, 'pb_ratio': 0.5668, 'industry_pb_ratio': 10.13291447368421}, {'symbol': 'BSX', 'industry': 'Health Care', 'pe_ratio': 13.4362, 'industry_pe_ratio': 44.649170103092786, 'pb_ratio': 3.4713, 'industry_pb_ratio': 10.13291447368421}, {'symbol': 'CI', 'industry': 'Health Care', 'pe_ratio': 12.9716, 'industry_pe_ratio': 44.649170103092786, 'pb_ratio': 1.4139, 'industry_pb_ratio': 10.132914473684

## Analysts Sentiment

In [74]:


print(price_target)
print(best_rank)
print(best_expert)

None
23
{'name': 'Louis Navellier ', 'firm': 'InvestorPlace', 'eUid': '39002a55faf604d6eccdafe2fdb139dcc85540e1', 'eTypeId': 3, 'expertImg': '22548_43E782ACB8', 'rankings': [{'period': 3, 'bench': 1, 'lRank': 5, 'gRank': 23, 'gRecs': 355, 'tRecs': 500, 'avgReturn': 0.255, 'stars': 5.0, 'tPos': 0.0}], 'ratings': [{'ratingId': 1, 'actionId': 8, 'date': '2020-05-15T00:00:00', 'd': '3m', 'url': 'https://investorplace.com/2020/05/7-a-rated-gold-stocks-to-buy-for-your-portfolio-hedge/', 'pos': 35, 'priceTarget': None, 'time': '2020-05-15T00:00:00', 'quote': {'title': '7 A-Rated Gold Stocks to Buy For Your Portfolio Hedge', 'date': '2020-05-15T00:00:00', 'quote': None, 'site': 'investorplace.com', 'link': 'https://investorplace.com/2020/05/7-a-rated-gold-stocks-to-buy-for-your-portfolio-hedge/'}, 'site': 'investorplace.com'}]}


In [75]:
final = {}


for s in underperforming_symbols:
    symbol = s['symbol']
    signal = None
    price_target = None
    last_updated = None

    req = rq.get(f'https://bank.tipranks.com/api/stocks/getData/{symbol}/?benchmark=1&period=3')
    if req.status_code != 200:
        print(f'Unable to get info for {symbol}')
        continue
        
    info = req.json()
    if len(info['experts']) < 10: 
        print(f'Skipped {symbol}. Not enough experts')
        continue

    # get signal
    ## 4 is to get the best consensus
    s['buy_consensus'] = info['consensuses'][4]['nB']
    s['hold_consensus'] = info['consensuses'][4]['nH']
    s['sell_consensus'] = info['consensuses'][4]['nS']
    s['consensus_rating'] = info['consensuses'][4]['rating']
    s['consensus_last_updated'] = last_consensus['d']

    s['price_target'] = info['ptConsensus'][0]['priceTarget'] if len(info['ptConsensus']) > 0 else None
    
    # current price
    s['price'] = info['prices'][-1]['p']
    s['price_date'] = info['prices'][-1]['d']

    
    print(f'Processed {symbol}')
    print(s)
print(final)



Processed ACB
{'symbol': 'ACB', 'industry': 'Health Care', 'pe_ratio': 3.1602, 'industry_pe_ratio': 44.649170103092786, 'pb_ratio': 0.3854, 'industry_pb_ratio': 10.13291447368421, 'buy_consensus': 0, 'hold_consensus': 1, 'sell_consensus': 0, 'consensus_rating': 3, 'consensus_last_updated': '27/8/20', 'price_target': 16.03, 'price': 9.82, 'price_date': '28/8/20'}
Processed ANTM
{'symbol': 'ANTM', 'industry': 'Health Care', 'pe_ratio': 12.2908, 'industry_pe_ratio': 44.649170103092786, 'pb_ratio': 2.0671, 'industry_pb_ratio': 10.13291447368421, 'buy_consensus': 2, 'hold_consensus': 2, 'sell_consensus': 0, 'consensus_rating': 4, 'consensus_last_updated': '27/8/20', 'price_target': 335.86, 'price': 277.28, 'price_date': '28/8/20'}
Skipped BIO. Not enough experts
Skipped BKD. Not enough experts
Processed BSX
{'symbol': 'BSX', 'industry': 'Health Care', 'pe_ratio': 13.4362, 'industry_pe_ratio': 44.649170103092786, 'pb_ratio': 3.4713, 'industry_pb_ratio': 10.13291447368421, 'buy_consensus': 8,

Skipped VEDL. Not enough experts
Skipped VRS. Not enough experts
Processed WRK
{'symbol': 'WRK', 'industry': 'Materials', 'pe_ratio': 10.1338, 'industry_pe_ratio': 35.737690566037735, 'pb_ratio': 0.6716, 'industry_pb_ratio': 3.1097270935960593, 'buy_consensus': 1, 'hold_consensus': 1, 'sell_consensus': 0, 'consensus_rating': 4, 'consensus_last_updated': '27/8/20', 'price_target': 31.0, 'price': 31.06, 'price_date': '28/8/20'}
Processed X
{'symbol': 'X', 'industry': 'Materials', 'pe_ratio': 2.0372, 'industry_pe_ratio': 35.737690566037735, 'pb_ratio': 0.4808, 'industry_pb_ratio': 3.1097270935960593, 'buy_consensus': 0, 'hold_consensus': 0, 'sell_consensus': 1, 'consensus_rating': 2, 'consensus_last_updated': '27/8/20', 'price_target': 3.0, 'price': 7.96, 'price_date': '28/8/20'}
Processed APLE
{'symbol': 'APLE', 'industry': 'Real Estate', 'pe_ratio': 16.2881, 'industry_pe_ratio': 49.43822243589744, 'pb_ratio': 0.6715, 'industry_pb_ratio': 2.375028813559322, 'buy_consensus': 0, 'hold_cons

Processed FL
{'symbol': 'FL', 'industry': 'Consumer Discretionary', 'pe_ratio': 14.0417, 'industry_pe_ratio': 30.180396551724137, 'pb_ratio': 1.2144, 'industry_pb_ratio': 12.130236, 'buy_consensus': 2, 'hold_consensus': 2, 'sell_consensus': 0, 'consensus_rating': 4, 'consensus_last_updated': '27/8/20', 'price_target': 33.75, 'price': 30.39, 'price_date': '28/8/20'}
Processed GCO
{'symbol': 'GCO', 'industry': 'Consumer Discretionary', 'pe_ratio': 9.462, 'industry_pe_ratio': 30.180396551724137, 'pb_ratio': 0.4852, 'industry_pb_ratio': 12.130236, 'buy_consensus': 0, 'hold_consensus': 1, 'sell_consensus': 0, 'consensus_rating': 3, 'consensus_last_updated': '27/8/20', 'price_target': 30.0, 'price': 19.07, 'price_date': '28/8/20'}
Skipped GHC. Not enough experts
Skipped GPI. Not enough experts
Processed GPS
{'symbol': 'GPS', 'industry': 'Consumer Discretionary', 'pe_ratio': 8.7312, 'industry_pe_ratio': 30.180396551724137, 'pb_ratio': 2.2034, 'industry_pb_ratio': 12.130236, 'buy_consensus': 1

Processed SAH
{'symbol': 'SAH', 'industry': 'Consumer Discretionary', 'pe_ratio': 8.3837, 'industry_pe_ratio': 30.180396551724137, 'pb_ratio': 2.6024, 'industry_pb_ratio': 12.130236, 'buy_consensus': 2, 'hold_consensus': 0, 'sell_consensus': 0, 'consensus_rating': 4, 'consensus_last_updated': '27/8/20', 'price_target': 37.5, 'price': 44.08, 'price_date': '28/8/20'}
Processed SIG
{'symbol': 'SIG', 'industry': 'Consumer Discretionary', 'pe_ratio': 10.1143, 'industry_pe_ratio': 30.180396551724137, 'pb_ratio': 0.6366, 'industry_pb_ratio': 12.130236, 'buy_consensus': 0, 'hold_consensus': 0, 'sell_consensus': 1, 'consensus_rating': 2, 'consensus_last_updated': '27/8/20', 'price_target': 5.0, 'price': 16.8, 'price_date': '28/8/20'}
Skipped SRI. Not enough experts
Processed TM
{'symbol': 'TM', 'industry': 'Consumer Discretionary', 'pe_ratio': 12.13, 'industry_pe_ratio': 30.180396551724137, 'pb_ratio': 0.9336, 'industry_pb_ratio': 12.130236, 'buy_consensus': 0, 'hold_consensus': 0, 'sell_consen

Skipped FBC. Not enough experts
Skipped FINV. Not enough experts
Processed FNB
{'symbol': 'FNB', 'industry': 'Financials', 'pe_ratio': 7.6122, 'industry_pe_ratio': 18.55852405498282, 'pb_ratio': 0.5034, 'industry_pb_ratio': 3.226949836065574, 'buy_consensus': 1, 'hold_consensus': 0, 'sell_consensus': 0, 'consensus_rating': 4, 'consensus_last_updated': '27/8/20', 'price_target': 8.0, 'price': 7.64, 'price_date': '28/8/20'}
Processed FSK
{'symbol': 'FSK', 'industry': 'Financials', 'pe_ratio': 7.9574, 'industry_pe_ratio': 18.55852405498282, 'pb_ratio': 0.6871, 'industry_pb_ratio': 3.226949836065574, 'buy_consensus': 0, 'hold_consensus': 1, 'sell_consensus': 0, 'consensus_rating': 3, 'consensus_last_updated': '27/8/20', 'price_target': 16.17, 'price': 16.07, 'price_date': '28/8/20'}
Skipped GCAP. Not enough experts
Skipped GNW. Not enough experts
Skipped GWB. Not enough experts
Skipped HCFT. Not enough experts
Processed HIG
{'symbol': 'HIG', 'industry': 'Financials', 'pe_ratio': 8.6354, 'i

Processed AL
{'symbol': 'AL', 'industry': 'Industrials', 'pe_ratio': 5.9383, 'industry_pe_ratio': 33.58921925925926, 'pb_ratio': 0.5974, 'industry_pb_ratio': 3.9346514950166114, 'buy_consensus': 2, 'hold_consensus': 0, 'sell_consensus': 0, 'consensus_rating': 4, 'consensus_last_updated': '27/8/20', 'price_target': 37.0, 'price': 32.31, 'price_date': '28/8/20'}
Skipped ARC. Not enough experts
Processed AYI
{'symbol': 'AYI', 'industry': 'Industrials', 'pe_ratio': 15.0176, 'industry_pe_ratio': 33.58921925925926, 'pb_ratio': 1.9371, 'industry_pb_ratio': 3.9346514950166114, 'buy_consensus': 1, 'hold_consensus': 1, 'sell_consensus': 0, 'consensus_rating': 4, 'consensus_last_updated': '27/8/20', 'price_target': 102.0, 'price': 110.94, 'price_date': '28/8/20'}
Skipped CAI. Not enough experts
Skipped CEA. Not enough experts
Skipped CIX. Not enough experts
Skipped CMRE. Not enough experts
Skipped CNHI. Not enough experts
Processed CPA
{'symbol': 'CPA', 'industry': 'Industrials', 'pe_ratio': 9.12

Unable to get info for LND
Skipped NGVC. Not enough experts
Skipped ODC. Not enough experts
Skipped PLAG. Not enough experts
Skipped RYCE. Not enough experts
Processed SJM
{'symbol': 'SJM', 'industry': 'Consumer Staples', 'pe_ratio': 16.807, 'industry_pe_ratio': 35.34212894736842, 'pb_ratio': 1.6015, 'industry_pb_ratio': 12.793407407407408, 'buy_consensus': 0, 'hold_consensus': 1, 'sell_consensus': 0, 'consensus_rating': 3, 'consensus_last_updated': '27/8/20', 'price_target': 123.5, 'price': 119.65, 'price_date': '28/8/20'}
Skipped SPB. Not enough experts
Processed TSN
{'symbol': 'TSN', 'industry': 'Consumer Staples', 'pe_ratio': 13.1207, 'industry_pe_ratio': 35.34212894736842, 'pb_ratio': 1.5987, 'industry_pb_ratio': 12.793407407407408, 'buy_consensus': 1, 'hold_consensus': 0, 'sell_consensus': 0, 'consensus_rating': 4, 'consensus_last_updated': '27/8/20', 'price_target': 67.0, 'price': 63.91, 'price_date': '28/8/20'}
Skipped USNA. Not enough experts
Skipped UVV. Not enough experts
Sk

Skipped TRT. Not enough experts
Skipped VHC. Not enough experts
Skipped WIT. Not enough experts
Processed XRX
{'symbol': 'XRX', 'industry': 'Information Technology', 'pe_ratio': 9.2135, 'industry_pe_ratio': 48.18109207920792, 'pb_ratio': 0.688, 'industry_pb_ratio': 8.83511012658228, 'buy_consensus': 0, 'hold_consensus': 1, 'sell_consensus': 0, 'consensus_rating': 3, 'consensus_last_updated': '27/8/20', 'price_target': 14.0, 'price': 19.53, 'price_date': '28/8/20'}
Skipped AROC. Not enough experts
Skipped ASC. Not enough experts
Skipped BCEI. Not enough experts
Processed BRY
{'symbol': 'BRY', 'industry': 'Energy', 'pe_ratio': 6.8868, 'industry_pe_ratio': 22.1664277486911, 'pb_ratio': 0.4633, 'industry_pb_ratio': 6.643146428571429, 'buy_consensus': 0, 'hold_consensus': 0, 'sell_consensus': 0, 'consensus_rating': 3, 'consensus_last_updated': '27/8/20', 'price_target': 7.0, 'price': 3.94, 'price_date': '28/8/20'}
Processed BSM
{'symbol': 'BSM', 'industry': 'Energy', 'pe_ratio': 9.9067, 'in

Processed IMO
{'symbol': 'IMO', 'industry': 'Energy', 'pe_ratio': 9.961, 'industry_pe_ratio': 22.1664277486911, 'pb_ratio': 0.7388, 'industry_pb_ratio': 6.643146428571429, 'buy_consensus': 0, 'hold_consensus': 0, 'sell_consensus': 0, 'consensus_rating': 3, 'consensus_last_updated': '27/8/20', 'price_target': None, 'price': 16.9, 'price_date': '28/8/20'}
Skipped INDO. Not enough experts
Processed INSW
{'symbol': 'INSW', 'industry': 'Energy', 'pe_ratio': 5.0969, 'industry_pe_ratio': 22.1664277486911, 'pb_ratio': 0.467, 'industry_pb_ratio': 6.643146428571429, 'buy_consensus': 0, 'hold_consensus': 0, 'sell_consensus': 0, 'consensus_rating': 3, 'consensus_last_updated': '27/8/20', 'price_target': 16.0, 'price': 16.93, 'price_date': '28/8/20'}
Processed KOS
{'symbol': 'KOS', 'industry': 'Energy', 'pe_ratio': 9.1892, 'industry_pe_ratio': 22.1664277486911, 'pb_ratio': 1.5353, 'industry_pb_ratio': 6.643146428571429, 'buy_consensus': 0, 'hold_consensus': 0, 'sell_consensus': 0, 'consensus_rating

Processed XEC
{'symbol': 'XEC', 'industry': 'Energy', 'pe_ratio': 8.2258, 'industry_pe_ratio': 22.1664277486911, 'pb_ratio': 1.6315, 'industry_pb_ratio': 6.643146428571429, 'buy_consensus': 0, 'hold_consensus': 0, 'sell_consensus': 0, 'consensus_rating': 3, 'consensus_last_updated': '27/8/20', 'price_target': 39.33, 'price': 28.72, 'price_date': '28/8/20'}
Skipped YPF. Not enough experts
Skipped CEPU. Not enough experts
Skipped EBR. Not enough experts
Skipped EDN. Not enough experts
Skipped ELP. Not enough experts
Skipped PAM. Not enough experts
Skipped AHC. Not enough experts
Skipped CHA. Not enough experts
Processed CHL
{'symbol': 'CHL', 'industry': 'Communication Services', 'pe_ratio': 9.2825, 'industry_pe_ratio': 33.734751515151515, 'pb_ratio': 0.892, 'industry_pb_ratio': 4.141546987951807, 'buy_consensus': 0, 'hold_consensus': 0, 'sell_consensus': 0, 'consensus_rating': 3, 'consensus_last_updated': '27/8/20', 'price_target': None, 'price': 36.37, 'price_date': '28/8/20'}
Skipped C

In [78]:
print(underperforming_symbols)
df = pd.DataFrame.from_dict(underperforming_symbols)
print(df)
df.to_csv('stocks_fund+sentiment_analysis.csv')

[{'symbol': 'ACB', 'industry': 'Health Care', 'pe_ratio': 3.1602, 'industry_pe_ratio': 44.649170103092786, 'pb_ratio': 0.3854, 'industry_pb_ratio': 10.13291447368421, 'buy_consensus': 0, 'hold_consensus': 1, 'sell_consensus': 0, 'consensus_rating': 3, 'consensus_last_updated': '27/8/20', 'price_target': 16.03, 'price': 9.82, 'price_date': '28/8/20'}, {'symbol': 'ANTM', 'industry': 'Health Care', 'pe_ratio': 12.2908, 'industry_pe_ratio': 44.649170103092786, 'pb_ratio': 2.0671, 'industry_pb_ratio': 10.13291447368421, 'buy_consensus': 2, 'hold_consensus': 2, 'sell_consensus': 0, 'consensus_rating': 4, 'consensus_last_updated': '27/8/20', 'price_target': 335.86, 'price': 277.28, 'price_date': '28/8/20'}, {'symbol': 'BIO', 'industry': 'Health Care', 'pe_ratio': 7.694, 'industry_pe_ratio': 44.649170103092786, 'pb_ratio': 2.0032, 'industry_pb_ratio': 10.13291447368421}, {'symbol': 'BKD', 'industry': 'Health Care', 'pe_ratio': 6.3043, 'industry_pe_ratio': 44.649170103092786, 'pb_ratio': 0.5668

# Get Stock Quotes

In [ ]:
with open('stocks_fundamentals-nyse.json') as f:
    new_stock_fundamentals = json.load(f)

In [10]:
stock_codes = []
for s in new_stock_fundamentals:
    stock_codes.append(s['General']['Code'])

#print(new_stock_fundamentals[0])
stock_codes.append('Date')

### Start New
#stock_quotes = pd.DataFrame(columns=stock_codes)
#stock_quotes = stock_quotes.set_index('Date')

### Get Quotes from existing ones
stock_quotes = pd.read_csv('stock_quotes-nyse.csv')
#print(new_stock_quotes)
stock_quotes = stock_quotes.set_index('Date')

print(stock_quotes.index)

Index(['2019-12-01', '2019-12-02', '2019-12-03', '2019-12-04', '2019-12-05',
       '2019-12-06', '2019-12-07', '2019-12-08', '2019-12-09', '2019-12-10',
       ...
       '2020-04-08', '2020-04-09', '2020-04-10', '2020-04-11', '2020-04-12',
       '2020-04-13', '2020-04-14', '2020-04-15', '2020-04-16', '2020-04-17'],
      dtype='object', name='Date', length=139)


In [92]:
## DEBUG CODE - SKIP THIS ##

#df = df.set_index('Code')
print(df)
quotes = []
for code in stock_quotes.columns:
    if code in df.index:
        quotes.append(df.loc[code]['Close'])
    else:
        quotes.append(None)
#print(len(quotes))
#print(len(stock_quotes.columns))
stock_quotes.loc[day] = quotes
print(stock_quotes)

          Ex        Date       Open       High        Low      Close  \
Code                                                                   
2875      US  2019-12-04  4710.0000  4750.0000  4685.0000  4725.0000   
4689      US  2019-12-04   377.0000   380.0000   372.0000   375.0000   
600598    US  2019-12-04     9.2400     9.4900     9.2300     9.4500   
6323      US  2019-12-04  3860.0000  3925.0000  3755.0000  3770.0000   
6976      US  2019-12-04  2853.0000  2863.0000  2777.0000  2853.0000   
...      ...         ...        ...        ...        ...        ...   
ZYNE      US  2019-12-04     6.0800     6.0800     5.8300     5.9700   
ZYRX      US  2019-12-04     0.0150     0.0250     0.0150     0.0250   
ZYXI      US  2019-12-04    10.8200    10.9600    10.2505    10.4000   
ZZZOF     US  2019-12-04     0.0114     0.0114     0.0114     0.0114   
2187850  NaN         NaN        NaN        NaN        NaN        NaN   

         Adjusted_close      Volume  
Code                     

In [11]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2015, 1, 1)
end_date = datetime.now().date() #date(2020, 4, 30)
for single_date in daterange(start_date, end_date):
    day = single_date.strftime("%Y-%m-%d")
    if day in stock_quotes.index:
        continue
    
    print(day)
    daily_quotes = rq.get('http://eodhistoricaldata.com/api/eod-bulk-last-day/US?api_token=&fmt=csv&date='+day)
    df = pd.read_csv(StringIO(daily_quotes.text))
    df = df.set_index('Code')
    #print(df)
    quotes = []
    for code in stock_quotes.columns:
        if code in df.index:
            quotes.append(df.loc[code]['Close'])
        else:
            quotes.append(None)
    stock_quotes.loc[day] = quotes



2015-01-01
2015-01-02
2015-01-03
2015-01-04


KeyboardInterrupt: 

In [40]:
stock_quotes.to_csv('stock_quotes-nyse.csv')

  # Preprocess stocks data

In [43]:
new_stock_quotes = pd.read_csv('stock_quotes-nyse.csv')
#print(new_stock_quotes)
new_stock_quotes = new_stock_quotes.set_index('Date')
#del new_stock_quotes['AAAP']

In [44]:
cols = new_stock_quotes.columns
for c in cols:
    quotes = new_stock_quotes[c]
    #quotes = quotes.reset_index(drop=True)
    #print(quotes)
    #quotes = quotes.dropna()
    non_nan_count = np.count_nonzero(~np.isnan(quotes))
    if non_nan_count < 75:
        del new_stock_quotes[c]
#print(new_stock_quotes)     

In [45]:
new_stock_quotes = new_stock_quotes.fillna(method='ffill')
#new_stock_quotes = new_stock_quotes.dropna()
#print(new_stock_quotes)

In [47]:
start_date = '2020-02-14'
today = datetime.now().date().strftime("%Y-%m-%d")

new_stock_quotes = new_stock_quotes.loc[start_date:today]
normalized = new_stock_quotes.apply(lambda x: (x / x.iloc[1]))
print(normalized.loc['2020-03-05'])
normalized = normalized.drop(['2020-03-05'])

#del normalized.loc['2020-03-05']
#normalized = normalized.dropna()
#print(normalized)

A       0.937544
AA      0.758775
AAMC    1.789753
AAN     0.616495
AAP     0.950071
          ...   
ZTO     1.098668
ZTR     0.926307
ZTS     0.968024
ZUO     0.754540
ZYME    0.814050
Name: 2020-03-05, Length: 2851, dtype: float64


# Calculate KPI's

In [48]:
all_quotes = new_stock_quotes
#all_quotes.index = pd.to_datetime(df.index)


### March Returns

In [49]:
start_date = '2020-2-29'
today = '2020-4-30'
march = all_quotes.loc[start_date:today]
#print(df)
march_returns = march.apply(lambda x: (x - x.shift(1))/x.shift(1))
#print(march_return)

### February Returns

In [50]:
start_date = '2020-1-31'
today = '2020-2-29'
feb = all_quotes.loc[start_date:today]
#print(df)
feb_returns = feb.apply(lambda x: (x - x.shift(1))/x.shift(1))


### January Returns


In [51]:
start_date = '2019-12-31'
today = '2020-01-31'
jan = all_quotes.loc[start_date:today]
#print(df)
jan_returns = jan.apply(lambda x: (x - x.shift(1))/x.shift(1))


### All Returns

In [52]:
all_returns = all_quotes.apply(lambda x: (x - x.shift(1))/x.shift(1))
#print(all_returns)

# Categorize Stocks

In [53]:
#print(new_stock_fundamentals[0])

## By Sector

In [8]:
by_sector = {}
for s in new_stock_fundamentals:
    sector = s["General"]["Sector"]
    if sector not in by_sector:
        by_sector[sector] = []
    by_sector[sector].append(s["General"]["Code"])

print(by_sector.keys())

dict_keys(['Healthcare', 'Basic Materials', 'Financial Services', 'Industrials', 'Consumer Cyclical', 'Real Estate', 'Industrial Goods', 'Consumer Defensive', 'Other', 'Technology', None, 'Energy', 'Utilities', '', 'Communication Services', 'Financial', 'Services', 'Consumer Goods'])


In [55]:
def avg_symbols_perf(symbols, data):
    #print(symbols)
    #print(data)
    df = pd.DataFrame(columns=symbols)
    df2 = data.filter(items=df.columns)#pd.merge(df,data, on=df.columns, how="inner")
    #print(df2)
    #print(df2.loc['2020-03-05'])
    df2 = df2.mean(axis=1)
    df2 = df2.replace([np.inf, -np.inf], np.nan)
    #df2 = df2.dropna()
    #print(df2)
    return df2


In [67]:
#avg_symbols_perf(by_sector["Technology"],all_returns)

returns_by_sector = pd.DataFrame(columns=by_sector.keys())
for k in by_sector:
    #print(by_sector[k])
    returns_by_sector[k] = avg_symbols_perf(by_sector[k],normalized)
    

returns_by_sector['Financial Services'] = None
returns_by_sector['Basic Materials'] = None

print(returns_by_sector)


            Healthcare Basic Materials Financial Services  Industrials  \
Date                                                                     
2020-02-14    1.000000            None               None     1.000000   
2020-02-15    1.000000            None               None     1.000000   
2020-02-16    1.000000            None               None     1.000000   
2020-02-17    1.000000            None               None     1.000000   
2020-02-18    0.999599            None               None     0.991560   
...                ...             ...                ...          ...   
2020-04-13    0.815564            None               None     0.681523   
2020-04-14    0.849333            None               None     0.694086   
2020-04-15    0.834420            None               None     0.672851   
2020-04-16    0.848777            None               None     0.661401   
2020-04-17    0.865315            None               None     0.686849   

            Consumer Cyclical  Real E

In [69]:
fig = go.Figure()
#print(returns_by_sector)
for k in returns_by_sector:
    print(k)
   # print(returns_by_sector[k])
    fig.add_trace(go.Scatter(
                    x=returns_by_sector.index,
                    y=returns_by_sector[k],
                    name=k,
                    #line_color='deepskyblue',
                    opacity=1))
fig.show()

Healthcare
Basic Materials
Financial Services
Industrials
Consumer Cyclical
Real Estate
Consumer Defensive
Other
Technology
Communication Services
Energy
Utilities

None
Services
Financial
Industrial Goods
Consumer Goods


In [84]:
for k in returns_by_sector:
    s = k
    v = returns_by_sector[k].iloc[-1]
    print(str(s)+','+str(v))

Healthcare,0.865314730834499
Basic Materials,None
Financial Services,None
Industrials,0.6868490621685942
Consumer Cyclical,0.6418370076542131
Real Estate,0.6066400178381547
Consumer Defensive,0.8610636177764972
Other,nan
Technology,0.7838697330658997
Communication Services,0.711144158786389
Energy,nan
Utilities,0.8081167585976297
,0.7441567743412411
None,0.6465662829143897
Services,0.8517313502207762
Financial,0.7889941093936134
Industrial Goods,0.6534320926637649
Consumer Goods,0.6309601567602874


# By Cig Industry

In [71]:
by_gicIndustry = {}
#print(new_stock_fundamentals[0])
for s in new_stock_fundamentals:
    industry = s["General"]["GicGroup"]
    if industry not in by_gicIndustry:
        by_gicIndustry[industry] = []
    by_gicIndustry[industry].append(s["General"]["Code"])

 #print(by_cigIndustry.keys())

In [75]:

returns_by_gicIndustry = pd.DataFrame(columns=by_gicIndustry.keys())
for k in returns_by_gicIndustry:
    #print(by_gicIndustry[k])
    returns_by_gicIndustry[k] = avg_symbols_perf(by_gicIndustry[k],normalized)

del returns_by_gicIndustry["Transportation"]
del returns_by_gicIndustry["Media"]
del returns_by_gicIndustry["Materials"]

#returns_by_gicIndustry['Financials'].loc['2020-03-05'] =1

In [76]:
fig = go.Figure()
#print(returns_by_sector)
for k in returns_by_gicIndustry:
    fig.add_trace(go.Scatter(
                    x=returns_by_gicIndustry.index,
                    y=returns_by_gicIndustry[k],
                    name=k,
                    #line_color='deepskyblue',
                    opacity=0.8))
fig.show()


In [25]:
print(by_gicIndustry["Diversified Financials"])

['AB', 'ABR', 'AC', 'ACRE', 'AI', 'AINC', 'AJX', 'ALLY', 'AMG', 'AMK', 'AMP', 'ANH', 'APAM', 'APO', 'ARES', 'ARI', 'ARR', 'ART', 'ASA', 'AXP', 'BAM', 'BBDC', 'BCF', 'BCSF', 'BEN', 'BK', 'BLK', 'BLX', 'BRK-B', 'BRKB', 'BSIG', 'BTZ', 'BX', 'BXMT', 'CHMI', 'CIM', 'CLNC', 'CMO', 'CNNE', 'CNS', 'CODI', 'COF', 'COHN', 'CS', 'CURO', 'DB', 'DFIN', 'DFS', 'DX', 'DXF', 'EARN', 'ECRO', 'EFC', 'ELVT', 'ENVA', 'EQH', 'EQS', 'EV', 'EVR', 'FCFS', 'FDS', 'FG', 'FII', 'FINV', 'FSK', 'GBL', 'GCAP', 'GDOT', 'GHL', 'GPMT', 'GS', 'GSBD', 'HCFT', 'HKIB', 'HLI', 'HTGC', 'ICE', 'IVR', 'IVZ', 'IX', 'JEF', 'JERT', 'JHG', 'JMP', 'JP', 'JT', 'KKR', 'KREF', 'LADR', 'LAZ', 'LC', 'LM', 'LTS', 'MAIN', 'MC', 'MCC', 'MCO', 'MDLY', 'MFA', 'MITT', 'MN', 'MS', 'MSCI', 'MVC', 'NLY', 'NMFC', 'NMR', 'NNI', 'NOAH', 'NRZ', 'OMF', 'ONDK', 'OPY', 'ORC', 'ORCC', 'PIPR', 'PJC', 'PJT', 'PMT', 'PPDF', 'PZN', 'QD', 'RC', 'RJF', 'RM', 'RWT', 'SAR', 'SC', 'SCHW', 'SCM', 'SCU', 'SF', 'SFE', 'SPGI', 'SRL', 'STT', 'STWD', 'SYF', 'TPVG', '

In [26]:
test = avg_symbols_perf(by_gicIndustry["Financials"], normalized)
#print(normalized)

KeyError: 'Financials'